In [1]:
!pip install -Uq dicomsdl

# nbdev requires jupyter, but we're already in a notebook environment, so we can install without dependencies
!pip install -U nbdev

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.1/64.1 kB 753.2 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.0/79.0 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.1/58.1 kB 3.1 MB/s eta 0:00:00


In [2]:
#|default_exp preprocess

In [3]:
#|export
from fastai.basics import *
from fastai.medical.imaging import *

import numpy as np
import pandas as pd
import cv2
import dicomsdl as dicom
from tqdm.notebook import tqdm, trange
from joblib import Parallel, delayed
import glob

In [4]:
#|export
class MammoPreprocessorBase():
    
    def __init__(self, img_path: str, image_size: tuple=(4096,2048), 
                 patch_size: tuple=(256,256), dir_name: str="Mammography_Dataset"):
    
        self.img_path = img_path
        self.image_size = image_size
        self.patch_size = patch_size
        os.makedirs(f"{dir_name}", exist_ok=True)
        self.save_path = os.path.join(os.getcwd(), dir_name)
        self.images = glob.glob(f"{img_path}/**/*.dcm", recursive=True)
    
    def preprocess_all(self, fformat: str, hist_eq: bool=True, n_jobs: int=-1):
        
        Parallel(n_jobs=n_jobs) \
            (delayed(self.preprocess_image) \
            (path, fformat, hist_eq) for path in tqdm(self.images, total=len(self.images)))
        print("Parallel preprocessing done!")
    
    def _hist_eq(self, img):
        img = self._convert_to_8bit(img)
        return cv2.equalizeHist(img)
    
    def _convert_to_8bit(self,img):
        return (img / img.max()*255).astype(np.uint8)
    
    def _croppad_to_width(self, img, size, mask=None):
        
        h, w  = img.shape
        
        if w < size[1]:
            img = cv2.copyMakeBorder(img, 0, 0, 0, size[1] - w, cv2.BORDER_CONSTANT, value=(0, 0, 0))
            if mask is not None:
                mask = cv2.copyMakeBorder(mask, 0, 0, 0, size[1] - w, cv2.BORDER_CONSTANT, value=(0, 0, 0))
        
        if w > size[1]:
            img = img[:, :size[1]]
            if mask is not None:
                mask = mask[:, :size[1]]
            
        return (img, mask) if mask is not None else img
    
    def _resize_to_height(self, img, size, mask=None):
        
        h,w = img.shape
        r = h/w
        new_size = (int(size[0]/r), size[0])
        
        img = cv2.resize(img, new_size)
        if mask is not None:
            mask = cv2.resize(mask, new_size, interpolation = cv2.INTER_NEAREST)
        
        return (img, mask) if mask is not None else img
    
    def _crop_roi(self, img, mask=None):
        
        bin_img = self._binarize(img)
        contour = self._find_contours(bin_img)
        
        x1, x2 = np.min(contour[:, :, 0]), np.max(contour[:, :, 0])
        y1, y2 = np.min(contour[:, :, 1]), np.max(contour[:, :, 1])
        
        return (img[y1:y2, x1:x2], mask[y1:y2, x1:x2]) if mask is not None else img[y1:y2, x1:x2] 
        
    def _remove_background(self, img, remove_wlines=False):
        #find a better solution to remove white lines
        
        bin_img = self._binarize(img)
        contour = self._find_contours(bin_img)
        
        mask = np.zeros(bin_img.shape, np.uint8)
        cv2.drawContours(mask, [contour], -1, 255, cv2.FILLED)
        
        if remove_wlines:
            white_lines_fix = (mask[:,-1]!=255).astype(np.uint8)[:,None]
            mask = (mask * white_lines_fix) / mask.max()
        
        return img * mask
    
    def _find_contours(self, bin_img):
    
        contours, _ = cv2.findContours(bin_img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
        contour = max(contours, key=cv2.contourArea)

        return contour
    
    def _binarize(self, img):
        
        binarized = (img > (img.max()*0.05)).astype("uint8")
        
        return binarized

    def _correct_side(self, img, mask=None):
        
        col_sums_split = np.array_split(np.sum(img, axis=0), 2)
        left_col_sum = np.sum(col_sums_split[0])
        right_col_sum = np.sum(col_sums_split[1])

        if left_col_sum > right_col_sum: 
            return (img, mask) if mask is not None else img
        else: 
            return (np.fliplr(img), np.fliplr(mask)) if mask is not None else np.fliplr(img)

In [5]:
#|export
class MammoPreprocessorCBISDDSM(MammoPreprocessorBase):
    
    def __init__(self, img_path: str, masks: str=None, 
               mammo_imgs_csv: str=None, masks_csv: str=None, case_desc_csv: str=None,
              image_size: tuple=(4096,2048), patch_size: tuple=(256,256), dir_name: str="CBIS_DDSM"):
    
        super().__init__(img_path, image_size, patch_size, dir_name)
        self.df = self._merge_dfs(mammo_imgs_csv, masks_csv, case_desc_csv)
        
    def preprocess_image(self, path:str, fformat: str="png", hist_eq: bool=True):
        
        img = dicom.open(path).pixelData()
        labels = self._combine_masks(path)
        
        img, labels = super()._correct_side(img, labels)
        img = super()._remove_background(img)
        img, labels = super()._crop_roi(img, labels)
        img, labels = super()._resize_to_height(img, self.image_size, labels)
        img, labels = super()._croppad_to_width(img, self.image_size, labels)
        if hist_eq:
            img = super()._hist_eq(img)
        else:
            img = super()._convert_to_8bit(img)
        
        self._save_image(img, path, fformat=fformat, mask=labels)
        
    def _save_image(self, img, path, fformat: str, mask=None):
        
        dir_path = self._create_save_path(path)
        fname = re.search("/.+/(.+_P_[0-9]+_.+?)/", path).group(1)
        
        fname_img = f"{fname}_mammo.{fformat}"
        save_path = os.path.join(dir_path, fname_img)
        cv2.imwrite(save_path, img)
        
        if mask is not None:
            fname_mask = f"{fname}_mask.png"
            save_path = os.path.join(dir_path, fname_mask)
            cv2.imwrite(save_path, mask)
        
    def _create_save_path(self, img_path):
 
        patient_folder = re.search("_(P_[0-9]+)_", img_path).group(1)
        
        save_path = os.path.join(self.save_path, patient_folder)
        os.makedirs(save_path, exist_ok=True)
        
        return save_path

    def _combine_masks(self, path):

        image_info = self.df.loc[self.df.full_img_fname==path]

        labels = 0
        for i in range(image_info.shape[0]):
            mask = image_info.iloc[i]
            mask_px = (dicom.open(mask.mask_fname).pixelData() / 255).astype(np.uint8)
            labels += mask_px * mask.pathology

        return labels
    
    def _merge_dfs(self, mammo_imgs_csv, masks_csv, case_desc_csv):
        
        df_full = pd.read_csv(mammo_imgs_csv)
        df_mask = pd.read_csv(masks_csv)
        df_mass = pd.read_csv(case_desc_csv)
        
        mass_type = df_full["PatientID"].str.split("_", n=1)[0][0]
        df_mass["PatientID"] = mass_type + "_" + df_mass.patient_id + "_" + df_mass["left or right breast"] + "_" + df_mass["image view"] + "_" + df_mass["abnormality id"].astype("str")
        df_mask = df_mask.loc[df_mask.SeriesDescription=="ROI mask images",].reset_index(drop=True)
        
        mass_cols_keep = ["PatientID", "pathology"]
        mask_cols_keep = ["PatientID", "fname"]
        full_cols_keep = ["PatientID", "fname"]

        df_mass = df_mass[mass_cols_keep]
        df_mask = df_mask[mask_cols_keep]
        df_full = df_full[full_cols_keep]
        
        df_mass.rename(columns={"PatientID": "PathologyID"}, inplace=True)
        df_mask.rename(columns={"PatientID": "PathologyID", "fname": "mask_fname"}, inplace=True)
        df_full.rename(columns={"PatientID": "ImageID", "fname": "full_img_fname"}, inplace=True)

        df_all = df_mass.merge(df_mask, on="PathologyID")
        df_all["ImageID"] = df_all.PathologyID.str.replace(r"_[0-9]$", "", regex=True)
        df_all = df_all.merge(df_full, on="ImageID", how="left")
        
        df_all["PatientID"] = df_all.PathologyID.str.extract("(P_[0-9]+)_", expand=False)
        df_all["pathology"] = df_all.pathology.str.replace("_.*","", regex=True)
        df_all["pathology"].replace({"BENIGN":1, "MALIGNANT":2}, inplace=True)

        df_all.sort_values(by="PatientID", ignore_index=True, inplace=True)
        
        return df_all

In [6]:
#|export
class MammoPreprocessorRSNA(MammoPreprocessorBase):
    
    def __init__(self, img_path: str, image_size: tuple=(4096,2048), 
                 patch_size: tuple=(256,256), dir_name: str="RSNA"):
    
        super().__init__(img_path, image_size, patch_size, dir_name)
        
    def preprocess_image(self, path:str, fformat: str="png", hist_eq: bool=True):
        
        scan, image = self._load_dicom(path)
        
        img = self._fix_photometric_inter(scan, img)
        img = self._windowing(scan, img)
        img = super()._correct_side(img)
        img = super()._remove_background(img)
        img = super()._crop_roi(img)
        img = super()._resize_to_height(img, self.image_size)
        img = super()._padresize_to_width(img, self.image_size)
        if hist_eq:
            img = super()._hist_eq(img)
        else:
            img = super()._convert_to_8bit(img)
        
        self._save_image(img, path, fformat=fformat)
        
    def _padresize_to_width(self, img, size):
        
        h, w  = img.shape
        
        if w < size[1]:
            img = cv2.copyMakeBorder(img, 0, 0, 0, size[1] - w, cv2.BORDER_CONSTANT, value=(0, 0, 0))
            
        if w > size[1]:
            img = cv2.resize(img, self.image_size)
            
        return img
    
    def _save_image(self, img, path, fformat: str):
        
        dir_path = self._create_save_path(path)
        fname = re.search("/([0-9]+).dcm$", path).group(1)
        
        fname_img = f"{fname}.{fformat}"
        save_path = os.path.join(dir_path, fname_img)
        cv2.imwrite(save_path, img)
    
    def _windowing(self, scan, img):
        
        function = scan.VOILUTFunction
        
        if type(scan.WindowWidth) == list:
            center = int(np.mean((scan.WindowCenter)))
            width = scan.WindowWidth[0]
        else:
            center = scan.WindowCenter
            width = scan.WindowWidth
        
        y_range = 2**scan.BitsStored - 1
        
        if function == 'SIGMOID':
            img = y_range / (1 + np.exp(-4 * (img - center) / width))
        
        else: # LINEAR
            
            center -= 0.5
            width -= 1
            
            below = img <= (center - width / 2)
            above = img > (center + width / 2)
            between = np.logical_and(~below, ~above)
            img[below] = 0
            img[above] = y_range
            img[between] = ((img[between] - center) / width + 0.5) * y_range
        
        return img
    
    def _fix_photometric_inter(self, scan, img):
        
        if scan.PhotometricInterpretation == "MONOCHROME1":
            img = img.max() - img
            
        return img
    
    def _create_save_path(self, img_path):
 
        patient_folder = re.search("/([0-9]+)/", img_path).group(1)
        
        save_path = os.path.join(self.save_path, patient_folder)
        os.makedirs(save_path, exist_ok=True)
        
        return save_path
    
    def _load_dicom(path: str):
        dcmfile = dicom.open(path)
        return dcmfile, dcmfile.pixelData()

In [7]:
%notebook -e preprocess.ipynb

from nbdev.export import nb_export
nb_export('preprocess.ipynb', '.')
!rm preprocess.ipynb

In [8]:
df = pd.read_csv("/kaggle/input/creating-a-metadata-dataframe/full_mammo_imgs.csv")